In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [8]:
a = np.load("data/test_carbon/100.npz")
data = a["n_s"]

In [10]:
kernel = np.load("data/kernel_generated2/Kernel_s_Carbon-loc-isoth-N2.xlsx.npy")

In [ ]:
def kernel_loading(pore_dist):
        return np.multiply(
            kernel,
            pore_dist[:, np.newaxis]
        ).sum(axis=0)